![ML](../5_Sources/Images/banner_ml.gif)

In [1]:
import pandas as pd
import numpy as np
import ast
import zipfile

>**`Seleccionar la tabla 'hotels'`**

In [ ]:
df_hotels = spark.sql("SELECT * FROM DataLakePG.hotels").toPandas()
df_hotels = df_hotels.loc[:, ['hotel_id','name','address','description','latitude','longitude','city_code','S2C','phones','web','email','stars_yelp','rating_gm']]
df_hotels['phones'] = df_hotels['phones'].apply(lambda x: ast.literal_eval(x) if x != None else x).apply(lambda x: x[0] if x != None else x).apply(pd.Series)['phoneNumber']
df_hotels.head(3)

In [87]:
df_hotels.info()

StatementMeta(, 81b95d61-9cf2-47c3-a35b-d827c8767044, 89, Finished, Available)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41627 entries, 0 to 41626
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   hotel_id     41627 non-null  int64  
 1   name         41627 non-null  object 
 2   address      41624 non-null  object 
 3   description  40302 non-null  object 
 4   latitude     41627 non-null  float64
 5   longitude    41627 non-null  float64
 6   city_code    41627 non-null  float64
 7   S2C          15857 non-null  float64
 8   phones       39505 non-null  object 
 9   web          10814 non-null  object 
 10  email        20233 non-null  object 
 11  stars_yelp   1593 non-null   float64
 12  rating_gm    1556 non-null   float64
dtypes: float64(6), int64(1), object(6)
memory usage: 4.1+ MB


>**`Seleccionar la tabla 'states'`**

In [88]:
df_states = spark.sql("SELECT * FROM DataLakePG.states").toPandas()
df_states = df_states.loc[:,['state_code','state_name']]
df_states.head(3)

StatementMeta(, 81b95d61-9cf2-47c3-a35b-d827c8767044, 90, Finished, Available)

,state_code,state_name
0,IL,ILLINOIS
1,CA,CALIFORNIA
2,MA,MASSACHUSETTS


>**`Seleccionar la tabla 'cities'`**

In [89]:
df_cities = spark.sql("SELECT * FROM DataLakePG.cities").toPandas()
df_cities['city_name'] = df_cities['city_name'].apply(lambda x: x.replace(',','').split()[0].title())
df_cities.head(3)

StatementMeta(, 81b95d61-9cf2-47c3-a35b-d827c8767044, 91, Finished, Available)

,city_code,state_code,city_name
0,2000001,FL,Terra
1,2000002,PA,King
2,2000003,FL,Saint


>**`Unir la tabla 'cities' y 'states'`**

In [90]:
df_states_cities = df_states.merge(df_cities, on='state_code', how='left').loc[:,['state_name','city_code','city_name']]
df_states_cities['city_code'] = df_states_cities['city_code'].astype('float64')
df_states_cities.head(3)

StatementMeta(, 81b95d61-9cf2-47c3-a35b-d827c8767044, 92, Finished, Available)

,state_name,city_code,city_name
0,ILLINOIS,2000012.0,Scott
1,ILLINOIS,2000024.0,Swansea
2,ILLINOIS,2000036.0,Ofallon


>**`Unir las tablas 'cities' y 'states' con 'hotels'`**

In [91]:
df_hotels = df_hotels.merge(df_states_cities, on='city_code', how='left')
df_hotels.head(3)

StatementMeta(, 81b95d61-9cf2-47c3-a35b-d827c8767044, 93, Finished, Available)

,hotel_id,name,address,description,latitude,longitude,city_code,S2C,phones,web,email,stars_yelp,rating_gm,state_name,city_name
0,40206,"The Eliza Jane, in The Unbound Collection by H...",315 Magazine St,The Eliza Jane Hotel is a 3.5 star hotel that ...,29.950025,-90.067891,104.0,3.0,+5042075071,None,None,4.0,4.2,LOUISIANA,New
1,160358,Holiday Inn Express Voorhees - Mt. Laurel,121 Laurel Oak Rd,This hotel enjoys a strategic setting just a s...,39.846067,-74.987809,2183.0,4.0,1-856-346-4500,None,None,4.0,3.8,NEW JERSEY,Lindenwold
2,167791,Best Western Plus South Coast Inn,5620 Calle Real,Welcome to Best Western Plus South Coast Inn!\...,34.441358,-119.819964,532.0,2.0,36113167559,None,None,3.5,NaN,CALIFORNIA,Goleta


>**`Seleccionar la tabla 'interest_points'`**

In [92]:
df_interest_points = spark.sql("SELECT * FROM DataLakePG.interest_points").toPandas()
df_interest_points = df_interest_points.loc[:,['hotel_id','interest_point_distance','poi_name']]
df_interest_points.head(3)

StatementMeta(, 81b95d61-9cf2-47c3-a35b-d827c8767044, 94, Finished, Available)

,hotel_id,interest_point_distance,poi_name
0,316491,40,Disneyland
1,316849,160,7th & Fig
2,317003,69202,Hoover Dam


>**`Unir la tabla 'interest_points' con 'hotels'`**

In [93]:
df_hotels = df_hotels.merge(df_interest_points, on='hotel_id', how='left')
df_hotels['poi_name'] = df_hotels['poi_name'].str.strip()
df_hotels.head(3)

StatementMeta(, 81b95d61-9cf2-47c3-a35b-d827c8767044, 95, Finished, Available)

,hotel_id,name,address,description,latitude,longitude,city_code,S2C,phones,web,email,stars_yelp,rating_gm,state_name,city_name,interest_point_distance,poi_name
0,40206,"The Eliza Jane, in The Unbound Collection by H...",315 Magazine St,The Eliza Jane Hotel is a 3.5 star hotel that ...,29.950025,-90.067891,104.0,3.0,+5042075071,None,None,4.0,4.2,LOUISIANA,New,NaN,NaN
1,160358,Holiday Inn Express Voorhees - Mt. Laurel,121 Laurel Oak Rd,This hotel enjoys a strategic setting just a s...,39.846067,-74.987809,2183.0,4.0,1-856-346-4500,None,None,4.0,3.8,NEW JERSEY,Lindenwold,NaN,NaN
2,167791,Best Western Plus South Coast Inn,5620 Calle Real,Welcome to Best Western Plus South Coast Inn!\...,34.441358,-119.819964,532.0,2.0,36113167559,None,None,3.5,NaN,CALIFORNIA,Goleta,4828,Goleta Tech Centre


>**`Llenar campos vacíos de las columnas 'stars_yelp' y 'rating_gm' de 'hotels'`**

In [94]:
df_hotels['stars_yelp'] = df_hotels['stars_yelp'].fillna(0)
df_hotels['rating_gm'] = df_hotels['rating_gm'].fillna(0)

StatementMeta(, 81b95d61-9cf2-47c3-a35b-d827c8767044, 96, Finished, Available)

>**`Crear una sola columna de puntuación (llamada 'stars') en 'hotels' a partir de 'stars_yelp' y 'rating_gm'`**

In [ ]:
df_hotels['stars'] = df_hotels['stars_yelp'].iloc[:]
for i in range(len(df_hotels['stars_yelp'])):
    if df_hotels['stars_yelp'][i] == 0.0 or df_hotels['rating_gm'][i] == 0.0:
        df_hotels['stars'][i] = df_hotels['stars_yelp'][i] + df_hotels['rating_gm'][i]
    elif df_hotels['stars_yelp'][i] > df_hotels['rating_gm'][i]:
        df_hotels['stars'][i] = df_hotels['stars_yelp'][i]
    else:
        df_hotels['stars'][i] = df_hotels['rating_gm'][i]
df_hotels = df_hotels.drop(columns=['stars_yelp', 'rating_gm'], axis=1)
df_hotels.iloc[:3]

In [104]:
df_hotels.info()

StatementMeta(, 81b95d61-9cf2-47c3-a35b-d827c8767044, 106, Finished, Available)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 57880 entries, 0 to 57879
Data columns (total 16 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   hotel_id                 57880 non-null  int64  
 1   name                     57880 non-null  object 
 2   address                  57877 non-null  object 
 3   description              56555 non-null  object 
 4   latitude                 57880 non-null  float64
 5   longitude                57880 non-null  float64
 6   city_code                57880 non-null  float64
 7   S2C                      26488 non-null  float64
 8   phones                   55758 non-null  object 
 9   web                      20349 non-null  object 
 10  email                    32811 non-null  object 
 11  state_name               57880 non-null  object 
 12  city_name                57880 non-null  object 
 13  interest_point_distance  25283 non-null  object 
 14  poi_name              

>**`Exportar dataframe de 'hotels'`**

In [96]:
df_hotels.to_csv("/lakehouse/default/Files/data/beta/Hotelbeds/df_hotels.csv", index=False)

StatementMeta(, 81b95d61-9cf2-47c3-a35b-d827c8767044, 98, Finished, Available)

# <h1 align=center>**`Endpoints`**</h1>

>**`Endpoint 'filters'`**

In [12]:
# Ruta del archivo ZIP
zip_file_path = '../2_Datasets/beta/hotelbeds/df_hotels.zip'

# Abre el archivo ZIP
with zipfile.ZipFile(zip_file_path, 'r') as zip_file:
    # Lista los archivos en el ZIP (puede haber múltiples archivos, no necesariamente uno)
    zip_file_contents = zip_file.namelist()
    
    # Supongamos que queremos leer el primer archivo CSV en el ZIP
    csv_file_name = zip_file_contents[0]
    
    # Extrae el archivo CSV del ZIP
    with zip_file.open(csv_file_name) as csv_file:
        # Lee el archivo CSV con Pandas
        df_hotels = pd.read_csv(csv_file)

In [10]:
#@app.get('')

def filters(state:str = '0', 
            city:str = '0',
            name_point:str = '0'):
    

    try:
        
        if state != '0':
            filter_state = df_hotels[df_hotels['state_name'] == state].sort_values(ascending=False, by=['stars'])
            if city != '0':
                filter_state = filter_state[filter_state['city_name'] == city].sort_values(ascending=False, by=['stars'])
                if name_point != '0':
                    filter_state = filter_state[filter_state['poi_name'] == name_point].sort_values(ascending=False, by=['stars'])
            return filter_state.iloc[:3,[11,12,1,15,2,3,4,5,8,9,13,14]]
        elif city != '0':
            filter_state = df_hotels[df_hotels['city_name'] == city].sort_values(ascending=False, by=['stars'])
            return filter_state.iloc[:3,[11,12,1,15,2,3,4,5,8,9,13,14]]
        elif name_point != '0':
            filter_state = df_hotels[df_hotels['poi_name'] == name_point].sort_values(ascending=False, by=['stars'])
            return filter_state.iloc[:3,[11,12,1,15,2,3,4,5,8,9,13,14]]         

    except:
        return 'INGRESAR UNA BÚSQUEDA'

In [17]:
filters(state='ARIZONA', city='Tucson', name_point='University of Arizona')

,state_name,city_name,name,stars,address,description,latitude,longitude,phones,web,interest_point_distance,poi_name
29899,ARIZONA,Tucson,Adobe Rose Inn,5.0,940 N Olsen Ave,NaN,32.233591,-110.940767,NaN,NaN,NaN,NaN
22919,ARIZONA,Tucson,Home2 Suites by Hilton Tucson Downtown,5.0,141 South Stone Ave,"We're located in vibrant downtown Tucson, home...",32.219860,-110.970294,+15202747400,https://www.hilton.com/en/hotels/tuscaht-home2...,NaN,NaN
29924,ARIZONA,Tucson,The Armory Park Inn,5.0,438 S 3rd Ave,NaN,32.216335,-110.964309,NaN,NaN,NaN,NaN
